### Python implementation of the algorithms mentioned in the book "Numerical Optimisation" by Nocedal and Wright, Springer

In [1]:
import numpy as np
import random

#### Algorithm 3.1: Backtracking Line Search

##### Used to analyse the Rosenbrock function, $f(x)$ = $100(x_2-x_1^2)^2+(1-x_1)^2$

In [9]:
x=np.array([-1.2, 1])
alpha=1
c=0.0001
rho=0.1

def f(x): return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

while True:
    B=np.array(([-400*(x[1]-3*x[0]**2), -400*x[0]], [-400*x[0], 200]))
    
    #B=np.eye(2)
    
    delta_f=np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
    print("B=",B)
    print("delta_f=",delta_f)
    p=-(np.linalg.inv(B).dot(delta_f))
    print("p=",p)
    
    x1=x+alpha*p
    L=c*alpha*((delta_f.T).dot(p))
    print("x1=", x1)
    print("L=", L)
    
    alpha=rho*alpha
    print("alpha=", alpha)
    
    if f(x1)>f(x)+L: break
    
    x=x1
alpha

B= [[1328.  480.]
 [ 480.  200.]]
delta_f= [-215.6  -88. ]
p= [0.025 0.38 ]
x1= [-1.175  1.38 ]
L= -0.0038829999999999963
alpha= 0.1
B= [[1104.75  470.  ]
 [ 470.    200.  ]]
delta_f= [-4.64375 -0.125  ]
p= [ 17.4      -40.889375]
x1= [ 0.565     -2.7089375]
L= -0.0007569007812492967
alpha= 0.010000000000000002


0.010000000000000002

#### Algorithm 3.2: Line Search ALgorithm

In [68]:
def LSA(alphaMax, x):
    alpha0=1
    i=1
    alpha1=random.random()
    c1=10**(-4)
    c2=0.9

    f0=f(x)
    g0=g(x)
    H0=H(x)
    B=np.copy(H0)

    alphaold=alpha0
    alphanew=alpha1
    fp=np.copy(f0)
    gp=np.copy(g0)
    
    while True:
        P=-(np.linalg.inv(B)).dot(gp)
        print(i)
        x1=x+alphanew*P
        xnew=np.copy(x1)
        fx=f(x1)
        gx=g(x1)
        Hx=H(x1)

        if (fx>f0+c1*alphanew*((g0.T).dot(P))) or ((i>1) and fx>=fp): return zoom(alphaold, alphanew, x, P)
        if abs((gx.T).dot(P))<=-c2*((g0.T).dot(P)): return alphanew
        if (gx.T).dot(P)>=0: return zoom(alphanew, alphaold, x, P)
        
        alphaold=alphanew
        fp=np.copy(fx)
        gp=np.copy(gx)
        Hp=np.copy(Hx)
        B=np.copy(Hp)
        
        alphanew+=random.uniform(alphanew, alphaMax)
        i+=1

LSA(1.3, np.copy([-3.5, -122.23]))

1
2


0.487412157195802

#### Algorithm 3.3: zoom

In [30]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))

def zoom(alpha1, alpha2, x, P):
    c1=10**(-4)
    c2=0.9
    f0=f(x)
    g0=g(x)
    
    while True:
        alphaj=(alpha1+alpha2)/2
        x1=x+alphaj*P
        fx=f(x1)
        gx=g(x1)
        x2=x+alpha1*P
        fl=f(x2)
        gl=g(x2)
        if fx>f0+c1*alphaj*((g0.T).dot(P)) or fx>=fl: alpha2=alphaj
        else:
            if abs((gx.T).dot(P))<=-c2*((g0.T).dot(P)): return alphaj
            if ((gx.T).dot(P))*(alpha2-alpha1)>=0:alpha2=aplha1
            alpha1=alpha2
x=np.array([1.2, 1.2])
zoom(0.085, 1.0, x, -np.linalg.inv(H(x)).dot(g(x)))

0.5425

#### Algorithm 4.2: Cauchy Point Algorithm

In [5]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))
def m(x, P): 
    B=H(x)
    p1=B.dot(P)
    return f(x)+(g(x).T).dot(P)+0.5*(P.T).dot(p1)

def CPA(x, Delta_max):

    delta=random.uniform(0, Delta_max)
    eta=random.uniform(0, 1/4)
    k=0
    while k<=5000:
        #print("k=", k)
        #print("x=", x)
        #print("delta=", delta)
        B=H(x)
        G=g(x)
        p1=B.dot(G)
        pd=(G.T).dot(p1)
        t=1 if pd<=0 else min(np.linalg.norm(G)**3/(delta*pd), 1)
        P=-(t*delta/np.linalg.norm(G))*G
        #print("P=", P)
        
        rho=(f(x)-f(x+P))/(m(x, np.zeros(2))-m(x, P))
        #print("rho=", rho)
        #print(P)
        if rho<1/4:
            delta=np.linalg.norm(P)/4
        else:
            if rho>3/4 and np.linalg.norm(P)==delta: delta=min(2*delta, Delta_max)
            else: delta=delta
        
        if rho>eta: x+=P
        else:x=x
        k+=1
    return x
        
CPA(np.array([-30.087, 0.563]), 2)

array([0.99866542, 0.99733061])

#### Algorithm 4.3: CG-Steihaug Algorithm

##### The trust region subproblem is as follows:
\begin{equation} 
min\space \space m(P)=f+g^TP+\frac{1}{2}P^TBP
\end{equation}
##### subject to:
\begin{equation} ||P||=\Delta \end{equation}

##### So,
\begin{equation}
\sqrt{(P_j^0+\tau d_j^0)^2 + (P_j^1+\tau d_j^1)^2+ ... +(P_j^n+\tau d_j^n)^2}=\Delta \\
\Rightarrow (P_j^0+\tau d_j^0)^2 + (P_j^1+\tau d_j^1)^2+ ... +(P_j^n+\tau d_j^n)^2=\Delta^2\\
\Rightarrow (P_j^0)^2+2P_j^0\tau d_j^0+(\tau d_j^0)^2+(P_j^1)^2+2P_j^1\tau d_j^1+(\tau d_j^1)^2+...+(P_j^n)^2+2P_j^n\tau d_j^n+(\tau d_j^n)^2=\Delta^2\\
\Rightarrow \tau^2 ((d_j^0)^2+(d_j^1)^2+...+(d_j^n)^2)+2\tau (P_j^0d_j^0+P_j^1d_j^1+...+P_j^nd_j^n)+(P_j^0)^2+(P_j^1)^2+...+(P_j^n)^2-\Delta^2=0\\
\Rightarrow \tau^2 ||d_j||^2+2\tau P_j^Td_j+||P_j||^2-\Delta^2=0\\
\Rightarrow \tau=\frac{-P_j^Td_j \frac{+}{} \sqrt{(P_j^Td_j)^2-||d_j||^2(||P_j||^2-\Delta^2)}}{||d_j||^2}
\end{equation}

In [3]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))

def CGS(Delta, x):
    epsilon=10**(-6)
    r=g(x)
    P=np.zeros(len(r))
    r0=np.copy(r)
    d=-np.copy(r)
    B=H(x)
    j=0
    if np.linalg.norm(r0)<epsilon: return P
    
    while True:
        p1=B.dot(d)
        alpha=(d.T).dot(p1)
        if alpha<=0:
            A=-(P.T).dot(d)
            C=((P.T).dot(d))**2
            D=np.linalg.norm(d)**2
            E=np.linalg.norm(P)**2-Delta**2
            alpha=(A+np.sqrt(C-D*E))/D

            return P + alpha*d
        else:
            alpha=((r.T).dot(r))/alpha
            P1=np.copy(P)
            P+=alpha*d
        
            if np.linalg.norm(P)>=Delta:
                A=-(P1.T).dot(d)
                C=((P1.T).dot(d))**2
                D=np.linalg.norm(d)**2
                E=np.linalg.norm(P1)**2-Delta**2
                alpha=(A+np.sqrt(C-D*E))/D

                return P1 + alpha*d
        
        r1=np.copy(r)
        r+=alpha*(B.dot(d))
        
        if np.linalg.norm(r)<epsilon*np.linalg.norm(r0): return P
        
        beta=((r.T).dot(r))/((r1.T).dot(r1))
        d1=np.copy(d)
        d=r+beta*d1
    
        print("P=",P)
        
        j+=1
        print(j)
        
CGS(1.5, np.array([1.2, -0.5]))
    

P= [-0.46233356  0.19255627]
1
P= [-0.77149824 -0.33422452]
2
P= [-0.91929185 -0.49909225]
3
P= [-1.0240319  -0.59281691]
4
P= [-1.10814938 -0.65741695]
5
P= [-1.17985734 -0.70631727]
6
P= [-1.24313913 -0.74543611]
7


array([-1.28705198, -0.7703877 ])

#### Dogleg implementation

In [4]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))
def m(x, P): 
    B=H(x)
    p1=B.dot(P)
    return f(x)+(g(x).T).dot(P)+0.5*(P.T).dot(p1)

def dogleg(Delta, Pg, Pn):
    if Delta<=np.linalg.norm(Pg): return (Delta/np.linalg.norm(Pg))*Pg
    elif Delta>=np.linalg.norm(Pn): return Pn
    else:
        A=np.linalg.norm(Pg)**2
        F=np.linalg.norm(Pn)**2
        C=np.linalg.norm(Pg-Pn)**2
        D=(A+F-C)/2
        E=(F-Delta**2)/(F-D+np.sqrt(D**2-A*F+Delta**2*C))
        return E*Pg+(1-E)*Pn

def DI(x, Delta_max):

    delta=random.uniform(0, Delta_max)
    eta=random.uniform(0, 1/4)
    k=0
    while k<=5000:
        #print("k=", k)
        #print("x=", x)
        #print("delta=", delta)
        B=H(x)
        G=g(x)
        p1=B.dot(G)
        Pg=-G*((G.T).dot(G))/((G.T).dot(p1))
        Pn=-np.linalg.inv(B).dot(G)
        P=dogleg(delta, Pg, Pn)
        #print("P=", P)
        
        rho=(f(x)-f(x+P))/(m(x, np.zeros(2))-m(x, P))
        #print("rho=", rho)
        #print(P)
        if rho<1/4:
            delta=np.linalg.norm(P)/4
        else:
            if rho>3/4 and np.linalg.norm(P)==delta: delta=min(2*delta, Delta_max)
            else: delta=delta
        
        if rho>eta: x+=P
        else:x=x
        k+=1
    return x    
        
DI(np.array([1.2, 1.2]), 2)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


array([1., 1.])

#### Algorithm 4.4: Exact Trust Region Algorithm

In [19]:
def ETR(Delta, B, G):
    Lambda=0
    epsilon=10**(-6)
    P=np.linalg.inv(B).dot(G)
    while abs(np.linalg.norm(P)-Delta)>epsilon and Lambda>=0:
        P1=-np.linalg.inv(B+Lambda*np.eye(len(P))).dot(P)
        Phi=np.linalg.norm(P)-Delta
        Phi1=((P.T).dot(P1))/np.linalg.norm(P)
        Lambda-=(Phi*np.linalg.norm(P))/(Phi1*Delta)
        P=-np.linalg.inv(B+Lambda*np.eye(len(G))).dot(G)
    if Lambda<0: P=-np.linalg.inv(B).dot(G)